<a href="https://colab.research.google.com/github/hana927/News-Summarization/blob/main/News_Summarization_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00


# Load Data

The CNN/DailyMail dataset consists of approximately 300,000 pairs of news articles and their corresponding summaries.

In [2]:
from datasets import load_dataset,load_metric

In [3]:
cnn_data = load_dataset("cnn_dailymail", cache_dir='data',version="3.0.0")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Let's dive into it and take a closer look:

In [4]:
 cnn_data

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Given infrastructure constraints we subset our dataset and limit training on only 10000 records.

In [6]:
cnn_data['train'] = cnn_data['train'].shuffle(seed=42).select(range(10000))
cnn_data['validation'] = cnn_data['validation'].shuffle(seed=42).select(range(2000))
cnn_data['test'] = cnn_data['test'].shuffle(seed=42).select(range(2000))

 This database contains three sub-databases, named 'train', 'validation', and 'test', respectively.Given a key (train, validation, or test) and an index, you can view the data.

In [5]:
cnn_data['train'][6]

{'article': 'BAGHDAD, Iraq (CNN) -- The women are too afraid and ashamed to show their faces or have their real names used. They have been driven to sell their bodies to put food on the table for their children -- for as little as $8 a day. Suha, 37, is a mother of three. She says her husband thinks she is cleaning houses when she leaves home. "People shouldn\'t criticize women, or talk badly about them," says 37-year-old Suha as she adjusts the light colored scarf she wears these days to avoid extremists who insist women cover themselves. "They all say we have lost our way, but they never ask why we had to take this path." A mother of three, she wears light makeup, a gold pendant of Iraq around her neck, and an unexpected air of elegance about her. "I don\'t have money to take my kid to the doctor. I have to do anything that I can to preserve my child, because I am a mother," she says, explaining why she prostitutes herself. Anger and frustration rise in her voice as she speaks. "No m

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(cnn_data["train"])

,article,highlights,id
0,"Breakthrough: The test, which had an 80 per cent success rate in trials, looks for a genetic 'signature that is linked to a high risk of cancer coming back . Scientists are developing a test to predict which prostate cancer patients are most likely to suffer a recurrence of the disease. It means doctors will be able to give those at high risk drugs to improve their chances. Men who have a low risk of the disease coming back after surgery and radiotherapy will be spared additional treatment. The new test, which had an 80 per cent success rate in a trial, looks for a genetic ‘signature’, or pattern, that is linked to a high risk of cancer coming back. The Canadian researchers will release details of their work today at a conference of the European Society for Radiotherapy and Oncology in Vienna. Professor Robert Bristow said between one-third and half of patients face a spread of cancer outside the prostate gland that is not detected in initial treatment. He said: ‘Men who fail treatment within two years may be at the highest risk of dying from their prostate cancer. ‘New tests are required that are better at predicting which patients will have cancer recur. ‘These men can then be offered additional treatments, such as chemotherapy and hormone therapy, that will combat the prostate cancer throughout their entire body . . . in order to improve their chances of survival.’ Prostate cancer is the most common cancer in men. In the UK, more than 41,000 men are diagnosed each year, leading to around 10,000 deaths annually. The new genetic test was developed by Professor Bristow, of the University of Toronto, and Dr Paul Boutros, of the Ontario Institute of Cancer Research. By analysing the DNA in tissue taken from a tumour before any treatment, it can predict whether surgery and radiotherapy will be successful for that patient. Professor Bristow said: ‘This is the first report of a test using this information derived from biopsy samples that can predict with close to 80 per cent accuracy which men are at high or low risk of prostate cancer recurring.’ The researchers plan further trials over the next two years to ensure it will work in hospitals. ‘If all goes well, this will lead to a new test for cancer patients that can be turned around in three days and will tell doctors which patients will do well with local treatment alone – surgery or radiotherapy – and which will need extra treatment,’ he said. Tumour: Doctors are keen to discover which cancer patients are likely to respond well to treatment . The researchers analysed DNA from 126 patients thought to be at intermediate risk, and followed their progress for eight years. They analysed each patient’s whole genome to identify areas with missing, extra, or irregular sections of genetic material. They found a pattern, or genetic signature, that is common to men at high risk of cancer recurring. They tested their findings on 150 further patients, with similar results. Professor Bristow said: ‘These results will enable us to develop a new way of personalising medicine.’ █ A prostate cancer treatment called brachytherapy, in which low-dose radioactive seeds are placed inside the prostate, could help half of men to retain their sex lives after treatment. Research shows this internal radiation is better than conventional radiotherapy, in which only one-third of men have their sexual function unaffected. It should be offered to all patients, said researcher Dr Renee Oismuller.",New test of genetic 'signature' had an 80 per cent success rate in a trial .\nOne-third to half of prostate cancer patients face disease spreading .\nProstate cancer is the most common cancer in British men .,25169aebf8de57e0ec5059250faf27d677dbfa38
1,"Eagle-eyed shoppers were able to purchase $400 PlayStations for just a quarter of the price after taking advantage of a loophole in Wal-Mart’s current price match promotion deal. On November 13 the world's largest retailer had announced that i

Since every article in this database has a manually written abstract, it makes it a good dataset for building an abstractive summarization model. Of course, if we only utilize the article portion, we can also use this database to build an extractive summarization model.